In [3]:
import random
import itertools

In [27]:
num_columns = 5
grid_size = 30
min_room_size = 3

roomspace = grid_size - (num_columns + 1)
def all_less_then(comb,n):
    for i in comb:
        if i<n:
            return False
    return True

def get_divisions(n, k):
    divisions = []
    combs = itertools.product(range(1, n), repeat=k)
    dic={}
    for comb in combs:
        if sum(comb) == n and all_less_then(comb,min_room_size):
            dic["".join(map(str,sorted(comb)))]=comb
    return list(dic.values())

divisions = get_divisions(roomspace, num_columns)


horizontal_split = list(random.choice(divisions))
vertical_splits = [list(random.choice(divisions)) for i in horizontal_split]
random.shuffle(horizontal_split)
for split in vertical_splits:
    random.shuffle(split)

horizontal_data = []
x_accum = 1
for part in horizontal_split:
    horizontal_data.append((x_accum,part))
    x_accum += part+1

vertical_data = []
for split in vertical_splits:
    y_accum = 1
    data = []
    for part in split:
        data.append((y_accum,part))
        y_accum += part+1
    vertical_data.append(data)
divisions = get_divisions(roomspace, num_columns)

In [189]:
num_rooms = 9

horizontal_split = list(random.choice(divisions))
vertical_splits = [list(random.choice(divisions)) for i in horizontal_split]
random.shuffle(horizontal_split)
for split in vertical_splits:
    random.shuffle(split)

horizontal_data = []
x_accum = 1
for part in horizontal_split:
    horizontal_data.append((x_accum,part))
    x_accum += part+1

vertical_data = []
for split in vertical_splits:
    y_accum = 1
    data = []
    for part in split:
        data.append((y_accum,part))
        y_accum += part+1
    vertical_data.append(data)


chosen_cells = 0
rooms_ixs = []

while True:
    x = random.choice(range(len(horizontal_split)))
    y = random.choice(range(len(vertical_splits[x])))
    ix = (x,y)
    if ix not in rooms_ixs:
        rooms_ixs.append(ix)
    if len(rooms_ixs) == num_rooms:
        break

chosen_rooms = []
for ix_x,ix_y in rooms_ixs:
    x,w = horizontal_data[ix_x]
    y,h = vertical_data[ix_x][ix_y]
    chosen_rooms.append((y,x,h,w))

def get_points(part):
    points = []
    y = part[0]
    x = part[1]
    for i in range(part[2]):
        for j in range(part[3]):
            points.append((y+i,x+j))
    return points
            
    
walkable_points = []
for w in chosen_rooms:
    walkable_points += get_points(w)
walkable_points = set(walkable_points)

def create_string(walkable):
    string = ""
    for i in range(grid_size):
        for j in range(grid_size):
            if (i,j) in walkable:
                string += " "
            else: string += "#"
        string += "\n"
    return string

print(create_string(walkable_points))    

##############################
###############   ############
###############   ############
###############   ############
###############   ############
##############################
#####         ############   #
#   #         ############   #
#   #         ############   #
#   #         ############   #
#####         ############   #
#   #         ############   #
#   ######################   #
#   ###############      #   #
###################      #####
#   ###############      #####
#   ##########################
#   ##########################
#   ###########   ############
#   ###########   ############
#   ###########   ############
#   ###########   ############
#   ###########   ############
#   ###########   ############
#   ##########################
##############################
###################      #####
###################      #####
###################      #####
##############################



In [158]:
def toG(y,x):
    return y-1,x-1
def toL(p):
    y,x = p
    return y+1,x+1
def get_room_center(room):
    y,x,h,w=room
    cy = y + (h//2)
    cx = x + (w//2)
    return (cy,cx)

In [190]:
import numpy as np
from scipy.spatial.distance import cdist

def get_distance(r1,r2):
    A = np.array(get_points(r1))
    B = np.array(get_points(r2))
    distances = cdist(A, B)
    min_distance = np.min(distances)
    return min_distance

distances = {}
for i,r1 in enumerate(chosen_rooms):
    for j,r2 in enumerate(chosen_rooms):
        if i<j:
            distances[(i,j)]=get_distance(r1,r2)


In [208]:
sorted_pairs=list(sorted(map(lambda x:(x[0],x[1]),distances.items()),key=lambda x:x[1]))

In [209]:
import networkx as nx
G=nx.grid_graph(dim=[grid_size-2,grid_size-2])

for e in G.edges(data=True):
    e[2]['weigth']=1
for n,d in G.nodes(data=True):
    d['taken']=False
for y,x in walkable_points:
    G.nodes[(y-1,x-1)]['taken']=True

def set_weights(G,taken,weigth):
    for node in taken:
        for k,v in G[node].items(): #edges connected to the node
            if G.nodes[k]['taken']==True: #ignore if the node on the other side is already taken
                  continue
            else:
                v['weigth']+=weigth
    return G
rooms_points =  map(lambda x:x[0],filter(lambda d: d[1]['taken'] == True, G.nodes(data=True)))
G = set_weights(G,rooms_points,100)

In [210]:
paths = {}
for pair,_ in sorted_pairs:
    r1 = chosen_rooms[pair[0]]
    r2 = chosen_rooms[pair[1]]
    c1=get_room_center(r1)
    c2=get_room_center(r2)
    path=nx.shortest_path(G, source=c1, target=c2, weight='weigth')
    paths[pair]=path
    for y,x in path:
        G.nodes[(y,x)]['taken']=True
    G=set_weights(G,path,50)

In [216]:
paths

{(0, 4): [(8, 2),
  (9, 2),
  (9, 3),
  (9, 4),
  (9, 5),
  (9, 6),
  (9, 7),
  (9, 8),
  (9, 9)],
 (0, 5): [(8, 2), (9, 2), (10, 2), (11, 2), (12, 2)],
 (1, 7): [(14, 22),
  (13, 22),
  (12, 22),
  (12, 23),
  (11, 23),
  (10, 23),
  (10, 24),
  (10, 25),
  (10, 26),
  (10, 27)],
 (4, 5): [(9, 9),
  (9, 8),
  (9, 7),
  (9, 6),
  (9, 5),
  (9, 4),
  (9, 3),
  (9, 2),
  (10, 2),
  (11, 2),
  (12, 2)],
 (5, 8): [(12, 2),
  (12, 1),
  (13, 1),
  (13, 2),
  (14, 2),
  (15, 2),
  (16, 2),
  (17, 2),
  (18, 2),
  (19, 2),
  (20, 2)],
 (2, 4): [(3, 16),
  (3, 15),
  (3, 14),
  (3, 13),
  (4, 13),
  (5, 13),
  (6, 13),
  (7, 13),
  (8, 13),
  (9, 13),
  (9, 12),
  (9, 11),
  (9, 10),
  (9, 9)],
 (1, 6): [(14, 22),
  (14, 21),
  (14, 20),
  (14, 19),
  (14, 18),
  (14, 17),
  (15, 17),
  (16, 17),
  (17, 17),
  (18, 17),
  (19, 17),
  (20, 17),
  (21, 17),
  (21, 16)],
 (3, 6): [(27, 22),
  (26, 22),
  (25, 22),
  (25, 21),
  (25, 20),
  (25, 19),
  (25, 18),
  (24, 18),
  (23, 18),
  (22, 18),

In [215]:
ix = -1
walkable_points1 = set(list(walkable_points) + list(map(toL,paths[sorted_pairs[ix][0]])))
print(create_string(walkable_points))

##############################
###############   ############
###############   ############
###############   ############
###############   ############
##############################
#####         ############   #
#   #         ############   #
#   #         ############   #
#   #         ############   #
#####         ############   #
#   #         ############   #
#   ######################   #
#   ###############      #   #
###################      #####
#   ###############      #####
#   ##########################
#   ##########################
#   ###########   ############
#   ###########   ############
#   ###########   ############
#   ###########   ############
#   ###########   ############
#   ###########   ############
#   ##########################
##############################
###################      #####
###################      #####
###################      #####
##############################



In [182]:
sorted_pairs

[((0, 1), 2.0),
 ((1, 3), 2.0),
 ((2, 5), 2.0),
 ((5, 6), 2.0),
 ((6, 7), 2.0),
 ((2, 8), 2.8284271247461903),
 ((3, 4), 2.8284271247461903),
 ((4, 6), 2.8284271247461903),
 ((4, 8), 2.8284271247461903),
 ((2, 6), 5.385164807134504),
 ((0, 3), 6.0),
 ((5, 8), 6.324555320336759),
 ((2, 4), 8.0),
 ((4, 5), 8.0),
 ((1, 4), 8.246211251235321),
 ((5, 7), 8.246211251235321),
 ((0, 7), 9.0),
 ((1, 6), 9.0),
 ((1, 7), 9.0),
 ((3, 6), 9.0),
 ((6, 8), 9.0),
 ((0, 6), 9.848857801796104),
 ((3, 7), 9.848857801796104),
 ((4, 7), 10.198039027185569),
 ((0, 4), 12.165525060596439),
 ((3, 8), 12.727922061357855),
 ((2, 7), 13.152946437965905),
 ((3, 5), 15.0),
 ((2, 3), 15.811388300841896),
 ((1, 5), 16.15549442140351),
 ((7, 8), 17.0),
 ((1, 8), 17.4928556845359),
 ((0, 5), 18.027756377319946),
 ((1, 2), 18.601075237738275),
 ((0, 8), 21.02379604162864),
 ((0, 2), 21.213203435596427)]